In [1]:
import pandas as pd
import numpy as np
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pickle
import yaml
from AE.AE import Autoencoder, GeneExpressionDataset

ModuleNotFoundError: No module named 'AE'

In [59]:
with open('Data/training_data.pkl','rb') as f:
    data=pickle.load(f)

In [60]:
X_train_tensor = data['X_train_tensor']
X_val_tensor = data['X_val_tensor']
input_dim = data['input_dim']
loader_workers=4

In [61]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [62]:
with open('optuna_parameter.yaml', "r") as f:
    optuna_parameters=yaml.safe_load(f)

In [63]:
optuna_parameters

{'batch_size': [64, 128, 256],
 'dropout_rate': [0, 0.5],
 'n_hidden_layers': [1, 3],
 'latent_dim': [16, 256],
 'lr': [1e-05, 0.001]}

In [67]:
def objective(trial: optuna.Trial, optuna_parameters, input_dim, loader_workers):
    # --- Hyperparameter Suggestions ---
    latent_dim = trial.suggest_int("latent_dim", *optuna_parameters["latent_dim"]) # Range for latent space size
    dropout_rate = trial.suggest_float("dropout_rate", *optuna_parameters["dropout_rate"]) # Range for dropout
    lr = trial.suggest_float('lr',*optuna_parameters["lr"]) # Learning rate
    batch_size = trial.suggest_categorical('batch_size',optuna_parameters["batch_size"]) # Batch size options
    n_hidden_layers = trial.suggest_int('n_hidden_layers',*optuna_parameters["n_hidden_layers"]) # Number hidden layers

    # Define hidden dimensions proportional to the previous layer
    hidden_dims = []
    current_dim_upper_bound = input_dim
    current_dim_lower_bound = latent_dim

    for i in range(n_hidden_layers):
         # Suggest dimensions between the current upper bound and latent_dim+buffer
         # relative to previous/next layer.
        if i == 0: # First hidden layer
             dim = trial.suggest_int(f'h_dim_{i}', int(latent_dim * 1.5), int(input_dim * 0.8))
        else: # Subsequent hidden layers
             # Ensure next layer is smaller than previous
             # Use the size of the previously suggested layer
             prev_dim = hidden_dims[-1]
             dim = trial.suggest_int(f'h_dim_{i}', int(latent_dim * 1.1), int(prev_dim * 0.9))
        hidden_dims.append(dim)

    # --- Model Training ---
    model = Autoencoder(input_dim, latent_dim, hidden_dims, dropout_rate).to(device)
    criterion = nn.MSELoss() # Mean Squared Error Loss for reconstruction
    optimizer = optim.Adam(model.parameters(), lr=lr) # Adam optimizer

    # Data Loaders
    train_dataset = GeneExpressionDataset(X_train_tensor)
    val_dataset = GeneExpressionDataset(X_val_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=loader_workers)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=loader_workers)

    # Training loop 
    N_EPOCHS = 20 # Few epochs to evaluate performances

    for epoch in range(N_EPOCHS):
        model.train()
        total_train_loss = 0
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            recon_x, _ = model(batch)
            loss = criterion(recon_x, batch)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        # Validation step
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                recon_x, _ = model(batch)
                loss = criterion(recon_x, batch)
                total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_loader)

        # --- Optuna Pruning ---
        trial.report(avg_val_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    # Return the final validation loss
    return avg_val_loss

In [68]:
study = optuna.create_study(direction='minimize', pruner=optuna.pruners.MedianPruner())

print("Starting Optuna optimization...")
# Run the optimization for a number of trials
# Adjust n_trials based on available time and computational resources
study.optimize(lambda trial: objective(trial, optuna_parameters, input_dim, loader_workers), n_trials=30)
print("\nOptimization finished.")
print("Best trial:")
trial = study.best_trial

print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-07-02 11:26:41,785] A new study created in memory with name: no-name-c30bfb47-6a36-4a4b-b157-d6320591b97c


Starting Optuna optimization...


[I 2025-07-02 11:27:12,346] Trial 0 finished with value: 0.6637172698974609 and parameters: {'latent_dim': 56, 'dropout_rate': 0.1123616935705582, 'lr': 0.00047764012193674823, 'batch_size': 256, 'n_hidden_layers': 1, 'h_dim_0': 936}. Best is trial 0 with value: 0.6637172698974609.
[I 2025-07-02 11:27:50,814] Trial 1 finished with value: 0.7155625522136688 and parameters: {'latent_dim': 83, 'dropout_rate': 0.07752974170260729, 'lr': 0.0009588025729706499, 'batch_size': 256, 'n_hidden_layers': 2, 'h_dim_0': 1453, 'h_dim_1': 362}. Best is trial 0 with value: 0.6637172698974609.
[I 2025-07-02 11:28:23,638] Trial 2 finished with value: 0.8383306860923767 and parameters: {'latent_dim': 59, 'dropout_rate': 0.4189875618161529, 'lr': 0.0005076863741872525, 'batch_size': 128, 'n_hidden_layers': 3, 'h_dim_0': 961, 'h_dim_1': 152, 'h_dim_2': 98}. Best is trial 0 with value: 0.6637172698974609.
[I 2025-07-02 11:29:02,508] Trial 3 finished with value: 0.6724217534065247 and parameters: {'latent_dim


Optimization finished.
Best trial:
  Value: 0.6441430374979973
  Params: 
    latent_dim: 146
    dropout_rate: 0.2590700565837698
    lr: 0.0002251207702163308
    batch_size: 64
    n_hidden_layers: 1
    h_dim_0: 1424


In [69]:
trial

FrozenTrial(number=7, state=TrialState.COMPLETE, values=[0.6441430374979973], datetime_start=datetime.datetime(2025, 7, 2, 11, 30, 19, 352509), datetime_complete=datetime.datetime(2025, 7, 2, 11, 31, 20, 631825), params={'latent_dim': 146, 'dropout_rate': 0.2590700565837698, 'lr': 0.0002251207702163308, 'batch_size': 64, 'n_hidden_layers': 1, 'h_dim_0': 1424}, user_attrs={}, system_attrs={}, intermediate_values={0: 0.7883007153868675, 1: 0.7474474608898163, 2: 0.7233505994081497, 3: 0.7049784362316132, 4: 0.6917879655957222, 5: 0.6809936910867691, 6: 0.6736280769109726, 7: 0.6675284057855606, 8: 0.6629672721028328, 9: 0.658944770693779, 10: 0.6559156179428101, 11: 0.652863010764122, 12: 0.6508064121007919, 13: 0.6500130370259285, 14: 0.647303007543087, 15: 0.6477040722966194, 16: 0.6464295759797096, 17: 0.6459380313754082, 18: 0.6444969326257706, 19: 0.6441430374979973}, distributions={'latent_dim': IntDistribution(high=256, log=False, low=16, step=1), 'dropout_rate': FloatDistribution